# SEResnet on CIFAR-10 image classification

In [1]:
#Importing libraries and required frameworks
import tensorflow as tf
from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#Loading the cifar-10 data
(X_train,Y_train),(X_test,Y_test)=datasets.cifar10.load_data()

In [ ]:
#Understanding the data and dimensions
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
Y_train=Y_train.reshape(-1) #Flatten the output into 1-D vector


In [ ]:
#Naming the classes 0-9 accordingly 
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
#To visualise the data
def plot_sample(X,Y,index):
    plt.figure(figsize=(15,2))
    plt.imshow(X[index])
    plt.xlabel(cifar10_classes[Y[index]])

In [ ]:
#check an example
plot_sample(X_train,Y_train,2)

In [ ]:
#scaling the data
X_train=X_train/255.0
X_test=X_test/255.0

Normal Neural Network

In [ ]:
nnn=models.Sequential([
    layers.Flatten(input_shape=(32,32,3)),
    layers.Dense(3000,activation='relu'),
    layers.Dense(1000,activation='relu'),
    layers.Dense(10,activation='softmax')
])
nnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
nnn.fit(X_train,Y_train,epochs=10)

In [ ]:
#checking the performance
nnn.evaluate(X_test,Y_test)


Accuracy of a normal Neural Network on training set=48.57

Accuracy of a normal Neural Network on test set=47.5

Convolutional Neural Network

In [ ]:
cnn=models.Sequential([
    #cnn
    layers.Conv2D(filters=32,activation='relu',kernel_size=(3,3),input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(filters=32,activation='relu',kernel_size=(3,3)),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax')
])
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
cnn.fit(X_train,Y_train,epochs=10)

In [ ]:
cnn.evaluate(X_test,Y_test)

Accuracy of normal CNN on training set=74.48%

Accuracy of normal CNN on test set=67.6%


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class ResNetBlock(tf.keras.Model):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        #Two convolutional layers
        self.conv1 = layers.Conv2D(filters=out_channels, kernel_size=3,strides=stride, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()

        self.conv2 = layers.Conv2D(filters=out_channels, kernel_size=3,strides=1, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()

        self.downsample = None
        #Downsample if it doesnt match
        if stride != 1 or in_channels != out_channels:
            self.downsample = tf.keras.Sequential([
                layers.Conv2D(filters=out_channels, kernel_size=1, strides=stride,padding='same', use_bias=False),
                layers.BatchNormalization()
            ])

    def call(self, x, training=False):
        identity = x  # Store the input for skip connection

        out = self.conv1(x)
        out = self.bn1(out, training=training)
        out = tf.nn.relu(out)

        out = self.conv2(out)
        out = self.bn2(out, training=training)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity #adding identity
        out = tf.nn.relu(out)
        return out


In [ ]:
class SEBlock(tf.keras.layers.Layer):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        #squeeze step 
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        #Excitation step used as a bottle neck layer
        self.fc1 = layers.Dense(channels // reduction, activation='relu')
        self.fc2 = layers.Dense(channels, activation='sigmoid')
        #scaling 
        self.reshape = layers.Reshape((1, 1, channels))

    def call(self, x):
        se = self.global_avg_pool(x)
        se = self.fc1(se)
        se = self.fc2(se)
        se = self.reshape(se)
        return x * se


In [ ]:
class SEResNetBlock(tf.keras.Model):
    def __init__(self, in_channels, out_channels, stride=1, reduction=16):
        super(SEResNetBlock, self).__init__()
        #Standard res block
        self.res_block = ResNetBlock(in_channels, out_channels, stride)
        #se block
        self.se_block = SEBlock(out_channels, reduction)

    def call(self, x, training=False):
        out = self.res_block(x, training=training)
        out = self.se_block(out)
        return out


In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(32, 32, 3)),

    # Initial conv layer (adjusted for CIFAR-10 input size)
    layers.Conv2D(64, kernel_size=3, strides=1, padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Stage 1
    SEResNetBlock(64, 64, stride=1),
    SEResNetBlock(64, 64, stride=1),

    # Stage 2
    SEResNetBlock(64, 128, stride=2),
    SEResNetBlock(128, 128, stride=1),

    # Stage 3
    SEResNetBlock(128, 256, stride=2),
    SEResNetBlock(256, 256, stride=1),

    # Stage 4
    SEResNetBlock(256, 512, stride=2),
    SEResNetBlock(512, 512, stride=1),

    # Classification head
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')  # 10 classes for CIFAR-10
])



In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
#Training the model
model.fit(
    X_train, Y_train,
    epochs=10,
)


Accuracy of training set=95.49%

Accuracy of test set=86.87%

JUPUDI ADARSH 

24CS01031